In [2]:
# === Load by filename -> compute agreement -> save & download summary ===
import os, json
import numpy as np
import pandas as pd
from pathlib import Path

# ----- SET YOUR FILE NAME HERE -----
FNAME = "yes_fake_overlap_cat_sub_filled.xlsx"  # or "yes_fake_overlap_cat_sub_filled.csv"

# Load file by name (from current Colab working dir)
if not os.path.exists(FNAME):
    raise FileNotFoundError(
        f"{FNAME} not found. "
        "Upload it to the Colab workspace or mount Drive.\n"
        "Tip: uncomment below to upload manually:\n"
        "# from google.colab import files\n# files.upload()"
    )

if FNAME.lower().endswith(".xlsx"):
    df = pd.read_excel(FNAME)
elif FNAME.lower().endswith(".csv"):
    df = pd.read_csv(FNAME)
else:
    raise ValueError("Please provide a .xlsx or .csv file")

# Detect count-matrix columns
cat_cols = [c for c in df.columns if c.startswith("count::cat::")]
sub_cols = [c for c in df.columns if c.startswith("count::sub::")]
count_cat = df[cat_cols].copy() if cat_cols else None
count_sub = df[sub_cols].copy() if sub_cols else None

# ---- Agreement functions ----
def krippendorff_alpha_nominal(count_matrix: pd.DataFrame) -> float:
    n_u = count_matrix.sum(axis=1).astype(int)   # ratings per item
    N = int(n_u.sum())                            # total ratings
    if N <= 1:
        return np.nan
    n_c = count_matrix.sum(axis=0).astype(int)   # totals per category

    Do_num, Do_den = 0, 0
    for i in range(count_matrix.shape[0]):
        ni = int(n_u.iloc[i])
        if ni >= 2:
            row = count_matrix.iloc[i, :].astype(int)
            Do_num += int((row * (ni - row)).sum())
            Do_den += ni * (ni - 1)
    if Do_den == 0:
        return np.nan
    Do = Do_num / Do_den

    if N * (N - 1) == 0:
        return np.nan
    De = 1.0 - ((n_c * (n_c - 1)).sum() / (N * (N - 1)))
    if De == 0:
        return np.nan

    return float(1.0 - (Do / De))

def fleiss_kappa_variable_n(count_matrix: pd.DataFrame) -> float:
    n_i = count_matrix.sum(axis=1).astype(int)
    mask = n_i >= 2
    if mask.sum() == 0:
        return np.nan
    cm = count_matrix.loc[mask]
    n_i = cm.sum(axis=1).astype(int)

    # Per-item agreement P_i
    Pi_num = (cm * (cm - 1)).sum(axis=1).astype(float)
    Pi_den = (n_i * (n_i - 1)).astype(float)
    Pi = Pi_num / Pi_den
    Pbar = Pi.mean()

    # Expected agreement from label proportions
    sum_over_items = cm.sum(axis=0).astype(float)
    total_ratings = float(n_i.sum())
    pj = sum_over_items / total_ratings
    Pe = float((pj ** 2).sum())
    if Pe == 1.0:
        return np.nan

    return float((Pbar - Pe) / (1.0 - Pe))

# Compute metrics
results = {"file": FNAME, "n_items_total": int(df.shape[0])}

if count_cat is not None:
    results["n_items_with_>=2_ratings_cat"] = int((count_cat.sum(axis=1) >= 2).sum())
    results["category"] = {
        "alpha_nominal": krippendorff_alpha_nominal(count_cat),
        "fleiss_kappa_variable_n": fleiss_kappa_variable_n(count_cat),
        "labels": [c.replace("count::cat::","") for c in count_cat.columns],
    }

if count_sub is not None:
    results["n_items_with_>=2_ratings_sub"] = int((count_sub.sum(axis=1) >= 2).sum())
    results["subcategory"] = {
        "alpha_nominal": krippendorff_alpha_nominal(count_sub),
        "fleiss_kappa_variable_n": fleiss_kappa_variable_n(count_sub),
        "labels": [c.replace("count::sub::","") for c in count_sub.columns],
    }

print(json.dumps(results, indent=2))

# Save & download summary JSON (optional)
OUT_JSON = Path("agreement_summary.json")
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

# To download in Colab:
# from google.colab import files
# files.download(str(OUT_JSON))


{
  "file": "yes_fake_overlap_cat_sub_filled.xlsx",
  "n_items_total": 667,
  "n_items_with_>=2_ratings_cat": 667,
  "category": {
    "alpha_nominal": 0.9348275654107517,
    "fleiss_kappa_variable_n": 0.9347949791934572,
    "labels": [
      "Business Model & Innovation",
      "Environment",
      "Human Capital",
      "Leadership & Governance",
      "Social Capital"
    ]
  },
  "n_items_with_>=2_ratings_sub": 667,
  "subcategory": {
    "alpha_nominal": 0.9173374506077113,
    "fleiss_kappa_variable_n": 0.917296119333015,
    "labels": [
      "Access & Affordability",
      "Air Quality",
      "Business Ethics",
      "Business Model Resilience",
      "Critical Incident Risk Management",
      "Customer Welfare",
      "Ecological Impacts",
      "Employee Engagement, Diversity & Inclusion",
      "Employee Health & Safety",
      "Energy Management",
      "GHG Emissions",
      "Human Rights & Community Relations",
      "Management of Legal & Regulatory Environment",
    